# NIP-11 & NIP-66 Test Notebook

This notebook tests the NIP-11 (Relay Information Document) and NIP-66 (Relay Monitoring) modules.

## Test Coverage

### NIP-11
- Fetch relay information document
- Parse and validate data fields
- SSL fallback handling
- Error handling for unreachable relays

### NIP-66
- DNS resolution (A, AAAA, CNAME, NS, PTR)
- SSL certificate validation and extraction
- HTTP headers capture (Server, X-Powered-By)
- Geolocation (requires GeoLite2-City database)
- Network/ASN info (requires GeoLite2-ASN database)
- RTT measurement (requires signing keys)

## Setup

In [3]:
import asyncio
import sys
from pathlib import Path

# Add src to path for imports
src_path = Path.cwd().parent / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print(f"Source path: {src_path}")
print(f"Python path includes src: {str(src_path) in sys.path}")

Source path: /Users/vincenzo/Documents/GitHub/BigBrotr/bigbrotr/src
Python path includes src: True


In [4]:
# Import NIP-11 and NIP-66 modules
from models.relay import Relay, NetworkType
from models.nips.nip11 import (
    Nip11,
    Nip11FetchMetadata,
    Nip11FetchData,
    Nip11FetchLogs,
)
from models.nips.nip66 import (
    Nip66,
    Nip66DnsMetadata,
    Nip66SslMetadata,
    Nip66HttpMetadata,
    Nip66GeoMetadata,
    Nip66NetMetadata,
    Nip66RttMetadata,
)

print("Imports successful!")

Imports successful!


In [5]:
# Test relays - mix of popular clearnet relays
TEST_RELAYS = [
    "wss://relay.damus.io",
    "wss://relay.nostr.band",
    "wss://nos.lol",
    "wss://relay.snort.social",
    "wss://nostr.wine",
]

# Create Relay objects
relays = [Relay(url) for url in TEST_RELAYS]
print(f"Created {len(relays)} test relays:")
for r in relays:
    print(f"  - {r.url} ({r.network.value})")

Created 5 test relays:
  - wss://relay.damus.io (clearnet)
  - wss://relay.nostr.band (clearnet)
  - wss://nos.lol (clearnet)
  - wss://relay.snort.social (clearnet)
  - wss://nostr.wine (clearnet)


---
# NIP-11 Tests

Test fetching and parsing relay information documents.

## Test 1: Basic NIP-11 Fetch

In [6]:
async def test_nip11_fetch_single(relay: Relay) -> Nip11FetchMetadata:
    """Fetch NIP-11 from a single relay."""
    print(f"\n{'='*60}")
    print(f"Fetching NIP-11 from: {relay.url}")
    print(f"{'='*60}")
    
    result = await Nip11FetchMetadata.fetch(relay, timeout=10.0)
    
    print(f"\nSuccess: {result.logs.success}")
    if not result.logs.success:
        print(f"Reason: {result.logs.reason}")
        return result
    
    # Print data fields
    data = result.data
    print(f"\nRelay Information:")
    print(f"  Name: {data.name}")
    print(f"  Description: {data.description[:100] if data.description else None}...")
    print(f"  Pubkey: {data.pubkey}")
    print(f"  Contact: {data.contact}")
    print(f"  Software: {data.software}")
    print(f"  Version: {data.version}")
    print(f"  Supported NIPs: {data.supported_nips}")
    
    if data.limitation:
        lim = data.limitation
        print(f"\nLimitations:")
        print(f"  Auth Required: {lim.auth_required}")
        print(f"  Payment Required: {lim.payment_required}")
        print(f"  Restricted Writes: {lim.restricted_writes}")
        print(f"  Max Message Length: {lim.max_message_length}")
        print(f"  Max Subscriptions: {lim.max_subscriptions}")
        print(f"  Min POW Difficulty: {lim.min_pow_difficulty}")
    
    return result

# Test with first relay
result = await test_nip11_fetch_single(relays[0])


Fetching NIP-11 from: wss://relay.damus.io

Success: True

Relay Information:
  Name: damus.io
  Description: Damus strfry relay...
  Pubkey: 32e1827635450ebb3c5a7d12c1f8e7b2b514439ac10a67eef3d9fd9c5c68e245
  Contact: jb55@jb55.com
  Software: git+https://github.com/hoytech/strfry.git
  Version: 1.0.4-1-g783f9ce8cc77
  Supported NIPs: [1, 2, 4, 9, 11, 22, 28, 40, 70, 77]

Limitations:
  Auth Required: None
  Payment Required: None
  Restricted Writes: None
  Max Message Length: 400000
  Max Subscriptions: 300
  Min POW Difficulty: None


In [20]:
result.to_dict()

{'data': {'name': 'damus.io',
  'description': 'Damus strfry relay',
  'icon': 'https://damus.io/img/logo.png',
  'pubkey': '32e1827635450ebb3c5a7d12c1f8e7b2b514439ac10a67eef3d9fd9c5c68e245',
  'contact': 'jb55@jb55.com',
  'software': 'git+https://github.com/hoytech/strfry.git',
  'version': '1.0.4-1-g783f9ce8cc77',
  'supported_nips': [1, 2, 4, 9, 11, 22, 28, 40, 70, 77],
  'limitation': {'max_message_length': 400000,
   'max_subscriptions': 300,
   'max_limit': 500},
  'fees': {}},
 'logs': {'success': True}}

## Test 2: NIP-11 Fetch Multiple Relays

In [7]:
async def test_nip11_fetch_multiple(relays: list[Relay]) -> dict[str, Nip11FetchMetadata]:
    """Fetch NIP-11 from multiple relays concurrently."""
    print(f"Fetching NIP-11 from {len(relays)} relays...\n")
    
    tasks = [Nip11FetchMetadata.fetch(r, timeout=10.0) for r in relays]
    results = await asyncio.gather(*tasks)
    
    # Summary table
    print(f"{'Relay':<40} {'Success':<8} {'Name':<25} {'NIPs'}")
    print("-" * 100)
    
    result_map = {}
    for relay, result in zip(relays, results):
        result_map[relay.url] = result
        success = "OK" if result.logs.success else "FAIL"
        name = (result.data.name or "-")[:24] if result.logs.success else result.logs.reason[:24] if result.logs.reason else "-"
        nips = len(result.data.supported_nips) if result.data.supported_nips else 0
        print(f"{relay.url:<40} {success:<8} {name:<25} {nips}")
    
    # Stats
    success_count = sum(1 for r in results if r.logs.success)
    print(f"\nSuccess rate: {success_count}/{len(relays)} ({100*success_count/len(relays):.1f}%)")
    
    return result_map

nip11_results = await test_nip11_fetch_multiple(relays)

Fetching NIP-11 from 5 relays...

Relay                                    Success  Name                      NIPs
----------------------------------------------------------------------------------------------------
wss://relay.damus.io                     OK       damus.io                  10
wss://relay.nostr.band                   FAIL     HTTP 502                  0
wss://nos.lol                            OK       nos.lol                   10
wss://relay.snort.social                 OK       Snort                     10
wss://nostr.wine                         OK       nostr.wine                9

Success rate: 4/5 (80.0%)


## Test 3: NIP-11 Using High-Level Nip11 Class

In [8]:
async def test_nip11_create(relay: Relay) -> Nip11:
    """Test the high-level Nip11.create() method."""
    print(f"Creating Nip11 for: {relay.url}")
    
    nip11 = await Nip11.create(relay, timeout=10.0)
    
    print(f"\nNip11 object created:")
    print(f"  relay: {nip11.relay.url}")
    print(f"  generated_at: {nip11.generated_at}")
    print(f"  fetch_metadata.logs.success: {nip11.fetch_metadata.logs.success}")
    
    if nip11.fetch_metadata.logs.success:
        print(f"  fetch_metadata.data.name: {nip11.fetch_metadata.data.name}")
        
        # Test serialization to RelayMetadata
        metadata_tuple = nip11.to_relay_metadata_tuple()
        print(f"\nRelayMetadata tuple:")
        print(f"  nip11_fetch type: {metadata_tuple.nip11_fetch.metadata_type.value}")
        print(f"  relay: {metadata_tuple.nip11_fetch.relay.url}")
    
    return nip11

nip11 = await test_nip11_create(relays[0])

Creating Nip11 for: wss://relay.damus.io

Nip11 object created:
  relay: wss://relay.damus.io
  generated_at: 1769681750
  fetch_metadata.logs.success: True
  fetch_metadata.data.name: damus.io

RelayMetadata tuple:
  nip11_fetch type: nip11_fetch
  relay: wss://relay.damus.io


## Test 4: NIP-11 Error Handling

In [9]:
async def test_nip11_error_handling():
    """Test NIP-11 fetch with invalid/unreachable relays."""
    test_cases = [
        ("wss://nonexistent.relay.invalid", "Non-existent domain"),
        ("wss://localhost:9999", "Connection refused"),
        ("wss://example.com", "Not a Nostr relay"),
    ]
    
    print("Testing error handling:\n")
    
    for url, description in test_cases:
        try:
            relay = Relay(url)
            result = await Nip11FetchMetadata.fetch(relay, timeout=5.0)
            
            print(f"{description}:")
            print(f"  URL: {url}")
            print(f"  Success: {result.logs.success}")
            print(f"  Reason: {result.logs.reason}")
            print()
        except Exception as e:
            print(f"{description}: Exception - {e}\n")

await test_nip11_error_handling()

Testing error handling:

Non-existent domain:
  URL: wss://nonexistent.relay.invalid
  Success: False
  Reason: Cannot connect to host nonexistent.relay.invalid:443 ssl:default [nodename nor servname provided, or not known]

Connection refused: Exception - Local addresses not allowed

Not a Nostr relay:
  URL: wss://example.com
  Success: False
  Reason: Invalid Content-Type: text/html



---
# NIP-66 Tests

Test relay monitoring capabilities.

## Test 5: DNS Resolution

In [10]:
async def test_nip66_dns(relay: Relay) -> Nip66DnsMetadata:
    """Test DNS resolution for a relay."""
    print(f"\n{'='*60}")
    print(f"DNS Resolution for: {relay.url}")
    print(f"{'='*60}")
    
    result = await Nip66DnsMetadata.dns(relay, timeout=10.0)
    
    print(f"\nSuccess: {result.logs.success}")
    if not result.logs.success:
        print(f"Reason: {result.logs.reason}")
        return result
    
    data = result.data
    print(f"\nDNS Data:")
    print(f"  IPv4 addresses: {data.dns_ips}")
    print(f"  IPv6 addresses: {data.dns_ips_v6}")
    print(f"  CNAME: {data.dns_cname}")
    print(f"  Reverse DNS: {data.dns_reverse}")
    print(f"  Nameservers: {data.dns_ns}")
    print(f"  TTL: {data.dns_ttl}")
    
    return result

dns_result = await test_nip66_dns(relays[0])


DNS Resolution for: wss://relay.damus.io

Success: True

DNS Data:
  IPv4 addresses: ['172.67.206.128', '104.21.61.55']
  IPv6 addresses: ['2606:4700:3030::ac43:ce80', '2606:4700:3033::6815:3d37']
  CNAME: None
  Reverse DNS: None
  Nameservers: ['elsa.ns.cloudflare.com', 'paul.ns.cloudflare.com']
  TTL: 299


## Test 6: SSL Certificate Check

In [11]:
async def test_nip66_ssl(relay: Relay) -> Nip66SslMetadata:
    """Test SSL certificate for a relay."""
    print(f"\n{'='*60}")
    print(f"SSL Certificate for: {relay.url}")
    print(f"{'='*60}")
    
    result = await Nip66SslMetadata.ssl(relay, timeout=10.0)
    
    print(f"\nSuccess: {result.logs.success}")
    if not result.logs.success:
        print(f"Reason: {result.logs.reason}")
        return result
    
    data = result.data
    print(f"\nSSL Data:")
    print(f"  Valid: {data.ssl_valid}")
    print(f"  Subject CN: {data.ssl_subject_cn}")
    print(f"  Issuer: {data.ssl_issuer}")
    print(f"  Issuer CN: {data.ssl_issuer_cn}")
    print(f"  Expires: {data.ssl_expires}")
    print(f"  Not Before: {data.ssl_not_before}")
    print(f"  Protocol: {data.ssl_protocol}")
    print(f"  Cipher: {data.ssl_cipher}")
    print(f"  Cipher Bits: {data.ssl_cipher_bits}")
    print(f"  SAN: {data.ssl_san[:3] if data.ssl_san else None}...")
    print(f"  Fingerprint: {data.ssl_fingerprint[:40] if data.ssl_fingerprint else None}...")
    
    return result

ssl_result = await test_nip66_ssl(relays[0])


SSL Certificate for: wss://relay.damus.io

Success: True

SSL Data:
  Valid: True
  Subject CN: None
  Issuer: None
  Issuer CN: None
  Expires: None
  Not Before: None
  Protocol: TLSv1.3
  Cipher: TLS_AES_256_GCM_SHA384
  Cipher Bits: 256
  SAN: None...
  Fingerprint: SHA256:27:92:D9:20:57:AF:A6:22:C5:97:65:...


## Test 7: HTTP Headers

In [12]:
async def test_nip66_http(relay: Relay) -> Nip66HttpMetadata:
    """Test HTTP headers extraction from WebSocket handshake."""
    print(f"\n{'='*60}")
    print(f"HTTP Headers for: {relay.url}")
    print(f"{'='*60}")
    
    result = await Nip66HttpMetadata.http(relay, timeout=10.0)
    
    print(f"\nSuccess: {result.logs.success}")
    if not result.logs.success:
        print(f"Reason: {result.logs.reason}")
        return result
    
    data = result.data
    print(f"\nHTTP Data:")
    print(f"  Server: {data.http_server}")
    print(f"  X-Powered-By: {data.http_powered_by}")
    
    return result

http_result = await test_nip66_http(relays[0])


HTTP Headers for: wss://relay.damus.io

Success: True

HTTP Data:
  Server: cloudflare
  X-Powered-By: None


## Test 8: Multiple NIP-66 Tests

In [13]:
async def test_nip66_multiple(relays: list[Relay]) -> dict[str, dict]:
    """Run DNS, SSL, HTTP tests on multiple relays."""
    print(f"Running NIP-66 tests on {len(relays)} relays...\n")
    
    results = {}
    
    for relay in relays:
        print(f"\nTesting: {relay.url}")
        
        # Run tests concurrently
        dns_task = Nip66DnsMetadata.dns(relay, timeout=10.0)
        ssl_task = Nip66SslMetadata.ssl(relay, timeout=10.0)
        http_task = Nip66HttpMetadata.http(relay, timeout=10.0)
        
        dns_result, ssl_result, http_result = await asyncio.gather(
            dns_task, ssl_task, http_task,
            return_exceptions=True
        )
        
        results[relay.url] = {
            "dns": dns_result,
            "ssl": ssl_result,
            "http": http_result,
        }
        
        # Print summary
        dns_ok = dns_result.logs.success if not isinstance(dns_result, Exception) else False
        ssl_ok = ssl_result.logs.success if not isinstance(ssl_result, Exception) else False
        http_ok = http_result.logs.success if not isinstance(http_result, Exception) else False
        
        print(f"  DNS: {'OK' if dns_ok else 'FAIL'}  SSL: {'OK' if ssl_ok else 'FAIL'}  HTTP: {'OK' if http_ok else 'FAIL'}")
    
    return results

nip66_results = await test_nip66_multiple(relays)

Running NIP-66 tests on 5 relays...


Testing: wss://relay.damus.io
  DNS: OK  SSL: OK  HTTP: OK

Testing: wss://relay.nostr.band
  DNS: OK  SSL: OK  HTTP: FAIL

Testing: wss://nos.lol
  DNS: OK  SSL: OK  HTTP: OK

Testing: wss://relay.snort.social
  DNS: OK  SSL: OK  HTTP: FAIL

Testing: wss://nostr.wine
  DNS: OK  SSL: OK  HTTP: OK


## Test 9: High-Level Nip66.create() (without RTT/Geo/Net)

In [14]:
async def test_nip66_create_basic(relay: Relay) -> Nip66:
    """Test Nip66.create() without RTT, Geo, Net (no databases/keys needed)."""
    print(f"\n{'='*60}")
    print(f"Creating Nip66 for: {relay.url}")
    print(f"{'='*60}")
    
    nip66 = await Nip66.create(
        relay,
        timeout=10.0,
        run_rtt=False,   # Requires keys
        run_geo=False,   # Requires GeoIP database
        run_net=False,   # Requires GeoIP database
        run_ssl=True,
        run_dns=True,
        run_http=True,
    )
    
    print(f"\nNip66 object created:")
    print(f"  relay: {nip66.relay.url}")
    print(f"  generated_at: {nip66.generated_at}")
    print(f"\nMetadata availability:")
    print(f"  rtt_metadata: {nip66.rtt_metadata is not None}")
    print(f"  ssl_metadata: {nip66.ssl_metadata is not None}")
    print(f"  geo_metadata: {nip66.geo_metadata is not None}")
    print(f"  net_metadata: {nip66.net_metadata is not None}")
    print(f"  dns_metadata: {nip66.dns_metadata is not None}")
    print(f"  http_metadata: {nip66.http_metadata is not None}")
    
    if nip66.ssl_metadata and nip66.ssl_metadata.logs.success:
        print(f"\nSSL valid: {nip66.ssl_metadata.data.ssl_valid}")
    
    if nip66.dns_metadata and nip66.dns_metadata.logs.success:
        print(f"DNS IPs: {nip66.dns_metadata.data.dns_ips}")
    
    # Test serialization
    metadata_tuple = nip66.to_relay_metadata_tuple()
    print(f"\nRelayMetadata tuple:")
    for field in ['nip66_rtt', 'nip66_ssl', 'nip66_geo', 'nip66_net', 'nip66_dns', 'nip66_http']:
        value = getattr(metadata_tuple, field)
        status = value.metadata_type.value if value else "None"
        print(f"  {field}: {status}")
    
    return nip66

nip66 = await test_nip66_create_basic(relays[0])


Creating Nip66 for: wss://relay.damus.io

Nip66 object created:
  relay: wss://relay.damus.io
  generated_at: 1769681784

Metadata availability:
  rtt_metadata: False
  ssl_metadata: True
  geo_metadata: False
  net_metadata: False
  dns_metadata: True
  http_metadata: True

SSL valid: True
DNS IPs: ['172.67.206.128', '104.21.61.55']

RelayMetadata tuple:
  nip66_rtt: None
  nip66_ssl: nip66_ssl
  nip66_geo: None
  nip66_net: None
  nip66_dns: nip66_dns
  nip66_http: nip66_http


## Test 10: Overlay Network Handling

In [15]:
async def test_overlay_network_handling():
    """Test that overlay-only tests raise appropriate errors."""
    print("Testing overlay network handling:\n")
    
    # Test with a Tor relay (without proxy - should fail)
    tor_url = "wss://relay.tor.example.onion"
    
    try:
        tor_relay = Relay(tor_url)
        print(f"Tor relay created: {tor_relay.url}")
        print(f"Network type: {tor_relay.network.value}")
        
        # SSL should fail for non-clearnet
        try:
            await Nip66SslMetadata.ssl(tor_relay, timeout=5.0)
            print("SSL test unexpectedly succeeded")
        except ValueError as e:
            print(f"SSL test correctly raised ValueError: {e}")
        
        # DNS should fail for non-clearnet  
        try:
            await Nip66DnsMetadata.dns(tor_relay, timeout=5.0)
            print("DNS test unexpectedly succeeded")
        except ValueError as e:
            print(f"DNS test correctly raised ValueError: {e}")
        
        # HTTP without proxy should fail
        try:
            await Nip66HttpMetadata.http(tor_relay, timeout=5.0)
            print("HTTP test unexpectedly succeeded")
        except ValueError as e:
            print(f"HTTP test correctly raised ValueError: {e}")
            
    except Exception as e:
        print(f"Error: {e}")

await test_overlay_network_handling()

Testing overlay network handling:

Tor relay created: ws://relay.tor.example.onion
Network type: tor
SSL test correctly raised ValueError: SSL test requires clearnet, got tor
DNS test correctly raised ValueError: DNS resolve requires clearnet, got tor
HTTP test correctly raised ValueError: overlay network tor requires proxy


---
# Summary Report

In [16]:
def print_summary():
    """Print test summary."""
    print("\n" + "="*60)
    print("TEST SUMMARY")
    print("="*60)
    
    print("\nNIP-11 Results:")
    for url, result in nip11_results.items():
        status = "OK" if result.logs.success else "FAIL"
        name = result.data.name if result.logs.success else "-"
        print(f"  [{status}] {url}: {name}")
    
    print("\nNIP-66 Results:")
    for url, tests in nip66_results.items():
        dns_ok = tests['dns'].logs.success if not isinstance(tests['dns'], Exception) else False
        ssl_ok = tests['ssl'].logs.success if not isinstance(tests['ssl'], Exception) else False
        http_ok = tests['http'].logs.success if not isinstance(tests['http'], Exception) else False
        print(f"  {url}")
        print(f"    DNS: {'OK' if dns_ok else 'FAIL'}  SSL: {'OK' if ssl_ok else 'FAIL'}  HTTP: {'OK' if http_ok else 'FAIL'}")

print_summary()


TEST SUMMARY

NIP-11 Results:
  [OK] wss://relay.damus.io: damus.io
  [FAIL] wss://relay.nostr.band: -
  [OK] wss://nos.lol: nos.lol
  [OK] wss://relay.snort.social: Snort
  [OK] wss://nostr.wine: nostr.wine

NIP-66 Results:
  wss://relay.damus.io
    DNS: OK  SSL: OK  HTTP: OK
  wss://relay.nostr.band
    DNS: OK  SSL: OK  HTTP: FAIL
  wss://nos.lol
    DNS: OK  SSL: OK  HTTP: OK
  wss://relay.snort.social
    DNS: OK  SSL: OK  HTTP: FAIL
  wss://nostr.wine
    DNS: OK  SSL: OK  HTTP: OK


---
# Optional: RTT Test with Keys

Uncomment and run if you have signing keys available.

In [17]:
# from nostr_sdk import Keys, EventBuilder, Filter, Kind, Tag
#
# async def test_nip66_rtt(relay: Relay, keys: Keys) -> Nip66RttMetadata:
#     """Test RTT (Round-Trip Time) measurement."""
#     print(f"\n{'='*60}")
#     print(f"RTT Test for: {relay.url}")
#     print(f"{'='*60}")
#     
#     event_builder = EventBuilder(Kind(30000), "nip66-test").tags([Tag.parse(["d", relay.url])])
#     read_filter = Filter().limit(1)
#     
#     result = await Nip66RttMetadata.rtt(
#         relay, keys, event_builder, read_filter, timeout=10.0
#     )
#     
#     print(f"\nLogs:")
#     print(f"  Open Success: {result.logs.open_success}")
#     print(f"  Open Reason: {result.logs.open_reason}")
#     print(f"  Read Success: {result.logs.read_success}")
#     print(f"  Read Reason: {result.logs.read_reason}")
#     print(f"  Write Success: {result.logs.write_success}")
#     print(f"  Write Reason: {result.logs.write_reason}")
#     
#     print(f"\nRTT Data:")
#     print(f"  Open RTT: {result.data.rtt_open} ms")
#     print(f"  Read RTT: {result.data.rtt_read} ms")
#     print(f"  Write RTT: {result.data.rtt_write} ms")
#     
#     return result
#
# # Generate test keys or load from environment
# keys = Keys.generate()
# rtt_result = await test_nip66_rtt(relays[0], keys)

---
# Optional: Geo/Net Tests with GeoIP Databases

Uncomment and run if you have GeoLite2 databases available.

In [18]:
# import geoip2.database
# from pathlib import Path
#
# async def test_nip66_geo_net(relay: Relay):
#     """Test Geolocation and Network info with GeoIP databases."""
#     city_db = Path("../static/GeoLite2-City.mmdb")
#     asn_db = Path("../static/GeoLite2-ASN.mmdb")
#     
#     if not city_db.exists() or not asn_db.exists():
#         print("GeoLite2 databases not found. Skipping geo/net tests.")
#         return
#     
#     city_reader = geoip2.database.Reader(str(city_db))
#     asn_reader = geoip2.database.Reader(str(asn_db))
#     
#     try:
#         print(f"\n{'='*60}")
#         print(f"Geo/Net Test for: {relay.url}")
#         print(f"{'='*60}")
#         
#         geo_result = await Nip66GeoMetadata.geo(relay, city_reader)
#         net_result = await Nip66NetMetadata.net(relay, asn_reader)
#         
#         if geo_result.logs.success:
#             print(f"\nGeo Data:")
#             print(f"  Country: {geo_result.data.geo_country}")
#             print(f"  City: {geo_result.data.geo_city}")
#             print(f"  Lat/Lon: {geo_result.data.geo_lat}, {geo_result.data.geo_lon}")
#             print(f"  Timezone: {geo_result.data.geo_tz}")
#             print(f"  Geohash: {geo_result.data.geohash}")
#         
#         if net_result.logs.success:
#             print(f"\nNet Data:")
#             print(f"  IP: {net_result.data.net_ip}")
#             print(f"  ASN: {net_result.data.net_asn}")
#             print(f"  ASN Org: {net_result.data.net_asn_org}")
#             print(f"  Network: {net_result.data.net_network}")
#     finally:
#         city_reader.close()
#         asn_reader.close()
#
# await test_nip66_geo_net(relays[0])